In [1]:
import sys
import io
import time
import requests
import datetime
import numpy as np
import pandas as pd

sys.path.append('../')
from src.util.functions import Functions

# Disable warning
pd.options.mode.chained_assignment = None 

level = 'regioni'

path_data = '../data'
path_csv = '{path_data}/csv'.format(path_data=path_data)

csv_out_name = '{path_csv}/covid19-{level}.csv'.format(path_csv=path_csv, level=level)
pop_regions_name = '{path_csv}/popolazione-regioni.csv'.format(path_csv=path_csv)
icu_regions_name = '{path_csv}/terapie-intensive.csv'.format(path_csv=path_csv)
regions_name = '{path_csv}/regioni.csv'.format(path_csv=path_csv)

In [2]:
url_cum = Functions.get_url_cum(level)
url_cum

'https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni.csv'

In [3]:
df_raw_r = pd.read_csv(url_cum)
df_raw_r

,data,stato,codice_regione,denominazione_regione,lat,long,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,...,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,note,ingressi_terapia_intensiva,note_test,note_casi
0,2020-02-24T18:00:00,ITA,13,Abruzzo,42.351222,13.398438,0,0,0,0,...,0,NaN,NaN,0,5,NaN,NaN,NaN,NaN,NaN
1,2020-02-24T18:00:00,ITA,17,Basilicata,40.639471,15.805148,0,0,0,0,...,0,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN
2,2020-02-24T18:00:00,ITA,18,Calabria,38.905976,16.594402,0,0,0,0,...,0,NaN,NaN,0,1,NaN,NaN,NaN,NaN,NaN
3,2020-02-24T18:00:00,ITA,15,Campania,40.839566,14.250850,0,0,0,0,...,0,NaN,NaN,0,10,NaN,NaN,NaN,NaN,NaN
4,2020-02-24T18:00:00,ITA,8,Emilia-Romagna,44.494367,11.341721,10,2,12,6,...,0,NaN,NaN,18,148,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6526,2020-12-30T17:00:00,ITA,19,Sicilia,38.115697,13.362357,1085,166,1251,32136,...,2381,NaN,NaN,92345,1211824,808160.0,NaN,12.0,NaN,NaN
6527,2020-12-30T17:00:00,ITA,9,Toscana,43.769231,11.255889,869,160,1029,8708,...,3655,NaN,NaN,119696,1872240,1057968.0,NaN,10.0,Positivi diagnosticati solo con test antigenic...,NaN
6528,2020-12-30T17:00:00,ITA,10,Umbria,43.106758,12.388247,256,44,300,3406,...,618,NaN,NaN,28660,500003,251329.0,NaN,8.0,NaN,NaN
6529,2020-12-30T17:00:00,ITA,2,Valle d'Aosta,45.737503,7.320149,72,2,74,332,...,379,NaN,NaN,7244,62480,37323.0,NaN,2.0,NaN,NaN


In [4]:
# Drop useless columns
cols_to_drop = ['stato', 'lat', 'long', 'note']
df_rd = df_raw_r.drop(cols_to_drop, axis=1)
df_rd

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,note_test,note_casi
0,2020-02-24T18:00:00,13,Abruzzo,0,0,0,0,0,0,0,0,0,NaN,NaN,0,5,NaN,NaN,NaN,NaN
1,2020-02-24T18:00:00,17,Basilicata,0,0,0,0,0,0,0,0,0,NaN,NaN,0,0,NaN,NaN,NaN,NaN
2,2020-02-24T18:00:00,18,Calabria,0,0,0,0,0,0,0,0,0,NaN,NaN,0,1,NaN,NaN,NaN,NaN
3,2020-02-24T18:00:00,15,Campania,0,0,0,0,0,0,0,0,0,NaN,NaN,0,10,NaN,NaN,NaN,NaN
4,2020-02-24T18:00:00,8,Emilia-Romagna,10,2,12,6,18,0,18,0,0,NaN,NaN,18,148,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6526,2020-12-30T17:00:00,19,Sicilia,1085,166,1251,32136,33387,-22,1084,56577,2381,NaN,NaN,92345,1211824,808160.0,12.0,NaN,NaN
6527,2020-12-30T17:00:00,9,Toscana,869,160,1029,8708,9737,-278,460,106304,3655,NaN,NaN,119696,1872240,1057968.0,10.0,Positivi diagnosticati solo con test antigenic...,NaN
6528,2020-12-30T17:00:00,10,Umbria,256,44,300,3406,3706,138,347,24336,618,NaN,NaN,28660,500003,251329.0,8.0,NaN,NaN
6529,2020-12-30T17:00:00,2,Valle d'Aosta,72,2,74,332,406,-7,14,6459,379,NaN,NaN,7244,62480,37323.0,2.0,NaN,NaN


In [5]:
last_date_str = df_rd.iloc[-1].data.split('T')[0]
last_date = datetime.datetime.strptime(last_date_str, '%Y-%m-%d')
prev_date = Functions.get_previous_date(last_date)
prev_date_str = datetime.datetime.strftime(prev_date, '%Y-%m-%d')
last_date_str, prev_date_str

('2020-12-30', '2020-12-29')

In [6]:
# Current state S
df_raw_r_1 = df_rd[df_rd.data.str.startswith(last_date_str)].sort_values(by='codice_regione').reset_index(drop=True)
df_raw_r_1

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,note_test,note_casi
0,2020-12-30T17:00:00,1,Piemonte,2957,192,3149,25810,28959,-424,1046,159609,7893,NaN,NaN,196461,1673896,981588.0,7.0,NaN,NaN
1,2020-12-30T17:00:00,2,Valle d'Aosta,72,2,74,332,406,-7,14,6459,379,NaN,NaN,7244,62480,37323.0,2.0,NaN,NaN
2,2020-12-30T17:00:00,3,Lombardia,3617,481,4098,48898,52996,-1422,1673,397010,25038,NaN,NaN,475044,4826709,2597894.0,21.0,NaN,NaN
3,2020-12-30T17:00:00,5,Veneto,2711,358,3069,86954,90023,-342,2986,152642,6410,NaN,NaN,249075,3276810,1208044.0,36.0,NaN,NaN
4,2020-12-30T17:00:00,6,Friuli Venezia Giulia,641,59,700,10947,11647,-130,803,35876,1627,NaN,NaN,49150,924819,338891.0,5.0,NaN,A seguito della periodica verifica dei casi se...
5,2020-12-30T17:00:00,7,Liguria,712,63,775,4853,5628,-131,415,51489,2880,NaN,NaN,59997,706883,322079.0,2.0,I 155891 test antigenici rapidi sono riferiti ...,NaN
6,2020-12-30T17:00:00,8,Emilia-Romagna,2641,230,2871,53186,56057,-755,1427,105656,7683,NaN,NaN,169396,2556405,1221526.0,19.0,NaN,NaN
7,2020-12-30T17:00:00,9,Toscana,869,160,1029,8708,9737,-278,460,106304,3655,NaN,NaN,119696,1872240,1057968.0,10.0,Positivi diagnosticati solo con test antigenic...,NaN
8,2020-12-30T17:00:00,10,Umbria,256,44,300,3406,3706,138,347,24336,618,NaN,NaN,28660,500003,251329.0,8.0,NaN,NaN
9,2020-12-30T17:00:00,11,Marche,468,62,530,10042,10572,443,585,28792,1557,NaN,NaN,40921,526915,311670.0,7.0,NaN,NaN


In [7]:
# State S - 1
df_raw_r_0 = df_rd[df_rd.data.str.startswith(prev_date_str)].sort_values(by='codice_regione').reset_index(drop=True)
df_raw_r_0

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,note_test,note_casi
0,2020-12-29T17:00:00,1,Piemonte,3029,196,3225,26158,29383,-920,840,158172,7860,NaN,NaN,195415,1664929,978418.0,7.0,NaN,NaN
1,2020-12-29T17:00:00,2,Valle d'Aosta,73,1,74,339,413,-81,0,6439,378,NaN,NaN,7230,62194,37216.0,0.0,Con riferimento alla nota inserita in data di ...,NaN
2,2020-12-29T17:00:00,3,Lombardia,3634,498,4132,50286,54418,-347,843,393995,24958,NaN,NaN,473371,4802831,2591103.0,45.0,NaN,NaN
3,2020-12-29T17:00:00,5,Veneto,2678,366,3044,87321,90365,-577,2655,149426,6298,NaN,NaN,246089,3258961,1203915.0,63.0,NaN,NaN
4,2020-12-29T17:00:00,6,Friuli Venezia Giulia,655,60,715,11062,11777,-335,493,34958,1612,NaN,NaN,48347,916889,337351.0,7.0,NaN,NaN
5,2020-12-29T17:00:00,7,Liguria,711,63,774,4985,5759,71,305,50956,2867,NaN,NaN,59582,701035,320126.0,7.0,I 151199 test antigenici rapidi sono riferiti ...,NaN
6,2020-12-29T17:00:00,8,Emilia-Romagna,2598,221,2819,53993,56812,-1486,894,103543,7614,NaN,NaN,167969,2538045,1216598.0,13.0,NaN,NaN
7,2020-12-29T17:00:00,9,Toscana,879,157,1036,8979,10015,-269,271,105589,3632,NaN,NaN,119236,1861004,1054622.0,6.0,Positivi diagnosticati solo con test antigenic...,NaN
8,2020-12-29T17:00:00,10,Umbria,263,38,301,3267,3568,10,161,24132,613,NaN,NaN,28313,496881,250303.0,8.0,NaN,NaN
9,2020-12-29T17:00:00,11,Marche,465,61,526,9603,10129,392,508,28661,1546,NaN,NaN,40336,519600,308324.0,7.0,NaN,NaN


In [8]:
# New df regions state S with Trentino-Alto Adige
df_r_1 = Functions.replace_bt_with_taa(df_raw_r_1)
df_r_1.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,note_test,note_casi
0,2020-12-30T17:00:00,1,Piemonte,2957,192,3149,25810,28959,-424,1046,159609,7893,NaN,NaN,196461,1673896,981588.0,7.0,NaN,NaN
1,2020-12-30T17:00:00,2,Valle d'Aosta,72,2,74,332,406,-7,14,6459,379,NaN,NaN,7244,62480,37323.0,2.0,NaN,NaN
2,2020-12-30T17:00:00,3,Lombardia,3617,481,4098,48898,52996,-1422,1673,397010,25038,NaN,NaN,475044,4826709,2597894.0,21.0,NaN,NaN
3,2020-12-30T17:00:00,4,Trentino-Alto Adige,548,67,615,11700,12315,-123,417,36775,1664,0.0,0.0,50754,807454,313197.0,6.0,0,NaN
4,2020-12-30T17:00:00,5,Veneto,2711,358,3069,86954,90023,-342,2986,152642,6410,NaN,NaN,249075,3276810,1208044.0,36.0,NaN,NaN
5,2020-12-30T17:00:00,6,Friuli Venezia Giulia,641,59,700,10947,11647,-130,803,35876,1627,NaN,NaN,49150,924819,338891.0,5.0,NaN,A seguito della periodica verifica dei casi se...
6,2020-12-30T17:00:00,7,Liguria,712,63,775,4853,5628,-131,415,51489,2880,NaN,NaN,59997,706883,322079.0,2.0,I 155891 test antigenici rapidi sono riferiti ...,NaN
7,2020-12-30T17:00:00,8,Emilia-Romagna,2641,230,2871,53186,56057,-755,1427,105656,7683,NaN,NaN,169396,2556405,1221526.0,19.0,NaN,NaN
8,2020-12-30T17:00:00,9,Toscana,869,160,1029,8708,9737,-278,460,106304,3655,NaN,NaN,119696,1872240,1057968.0,10.0,Positivi diagnosticati solo con test antigenic...,NaN
9,2020-12-30T17:00:00,10,Umbria,256,44,300,3406,3706,138,347,24336,618,NaN,NaN,28660,500003,251329.0,8.0,NaN,NaN


In [9]:
# New df regions state S - 1 with Trentino-Alto Adige
df_r_0 = Functions.replace_bt_with_taa(df_raw_r_0)
df_r_0.sort_values(by='codice_regione').reset_index()

,index,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,...,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,note_test,note_casi
0,0,2020-12-29T17:00:00,1,Piemonte,3029,196,3225,26158,29383,-920,...,158172,7860,NaN,NaN,195415,1664929,978418.0,7.0,NaN,NaN
1,1,2020-12-29T17:00:00,2,Valle d'Aosta,73,1,74,339,413,-81,...,6439,378,NaN,NaN,7230,62194,37216.0,0.0,Con riferimento alla nota inserita in data di ...,NaN
2,2,2020-12-29T17:00:00,3,Lombardia,3634,498,4132,50286,54418,-347,...,393995,24958,NaN,NaN,473371,4802831,2591103.0,45.0,NaN,NaN
3,19,2020-12-29T17:00:00,4,Trentino-Alto Adige,516,68,584,11854,12438,10,...,36244,1655,0.0,0.0,50337,802451,312123.0,7.0,0,NaN
4,3,2020-12-29T17:00:00,5,Veneto,2678,366,3044,87321,90365,-577,...,149426,6298,NaN,NaN,246089,3258961,1203915.0,63.0,NaN,NaN
5,4,2020-12-29T17:00:00,6,Friuli Venezia Giulia,655,60,715,11062,11777,-335,...,34958,1612,NaN,NaN,48347,916889,337351.0,7.0,NaN,NaN
6,5,2020-12-29T17:00:00,7,Liguria,711,63,774,4985,5759,71,...,50956,2867,NaN,NaN,59582,701035,320126.0,7.0,I 151199 test antigenici rapidi sono riferiti ...,NaN
7,6,2020-12-29T17:00:00,8,Emilia-Romagna,2598,221,2819,53993,56812,-1486,...,103543,7614,NaN,NaN,167969,2538045,1216598.0,13.0,NaN,NaN
8,7,2020-12-29T17:00:00,9,Toscana,879,157,1036,8979,10015,-269,...,105589,3632,NaN,NaN,119236,1861004,1054622.0,6.0,Positivi diagnosticati solo con test antigenic...,NaN
9,8,2020-12-29T17:00:00,10,Umbria,263,38,301,3267,3568,10,...,24132,613,NaN,NaN,28313,496881,250303.0,8.0,NaN,NaN


In [10]:
df_r = df_r_1.copy()
# Merge with population data
df_pop_r = pd.read_csv(pop_regions_name)
df_m_r_1 = df_r.merge(df_pop_r, on='denominazione_regione')
df_m_r_1.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,note_test,note_casi,popolazione
0,2020-12-30T17:00:00,1,Piemonte,2957,192,3149,25810,28959,-424,1046,...,7893,NaN,NaN,196461,1673896,981588.0,7.0,NaN,NaN,4341375
1,2020-12-30T17:00:00,2,Valle d'Aosta,72,2,74,332,406,-7,14,...,379,NaN,NaN,7244,62480,37323.0,2.0,NaN,NaN,125501
2,2020-12-30T17:00:00,3,Lombardia,3617,481,4098,48898,52996,-1422,1673,...,25038,NaN,NaN,475044,4826709,2597894.0,21.0,NaN,NaN,10103969
3,2020-12-30T17:00:00,4,Trentino-Alto Adige,548,67,615,11700,12315,-123,417,...,1664,0.0,0.0,50754,807454,313197.0,6.0,0,NaN,1074819
4,2020-12-30T17:00:00,5,Veneto,2711,358,3069,86954,90023,-342,2986,...,6410,NaN,NaN,249075,3276810,1208044.0,36.0,NaN,NaN,4907704
5,2020-12-30T17:00:00,6,Friuli Venezia Giulia,641,59,700,10947,11647,-130,803,...,1627,NaN,NaN,49150,924819,338891.0,5.0,NaN,A seguito della periodica verifica dei casi se...,1211357
6,2020-12-30T17:00:00,7,Liguria,712,63,775,4853,5628,-131,415,...,2880,NaN,NaN,59997,706883,322079.0,2.0,I 155891 test antigenici rapidi sono riferiti ...,NaN,1543127
7,2020-12-30T17:00:00,8,Emilia-Romagna,2641,230,2871,53186,56057,-755,1427,...,7683,NaN,NaN,169396,2556405,1221526.0,19.0,NaN,NaN,4467118
8,2020-12-30T17:00:00,9,Toscana,869,160,1029,8708,9737,-278,460,...,3655,NaN,NaN,119696,1872240,1057968.0,10.0,Positivi diagnosticati solo con test antigenic...,NaN,3722729
9,2020-12-30T17:00:00,10,Umbria,256,44,300,3406,3706,138,347,...,618,NaN,NaN,28660,500003,251329.0,8.0,NaN,NaN,880285


In [11]:
# Merge with icu data
df_icu_r = pd.read_csv(icu_regions_name)
df_m_r_2 = df_m_r_1.merge(df_icu_r, on='denominazione_regione')
df_m_r_2.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,note_test,note_casi,popolazione,totale_posti_terapia_intensiva
0,2020-12-30T17:00:00,1,Piemonte,2957,192,3149,25810,28959,-424,1046,...,NaN,NaN,196461,1673896,981588.0,7.0,NaN,NaN,4341375,575
1,2020-12-30T17:00:00,2,Valle d'Aosta,72,2,74,332,406,-7,14,...,NaN,NaN,7244,62480,37323.0,2.0,NaN,NaN,125501,20
2,2020-12-30T17:00:00,3,Lombardia,3617,481,4098,48898,52996,-1422,1673,...,NaN,NaN,475044,4826709,2597894.0,21.0,NaN,NaN,10103969,1036
3,2020-12-30T17:00:00,4,Trentino-Alto Adige,548,67,615,11700,12315,-123,417,...,0.0,0.0,50754,807454,313197.0,6.0,0,NaN,1074819,106
4,2020-12-30T17:00:00,5,Veneto,2711,358,3069,86954,90023,-342,2986,...,NaN,NaN,249075,3276810,1208044.0,36.0,NaN,NaN,4907704,1016
5,2020-12-30T17:00:00,6,Friuli Venezia Giulia,641,59,700,10947,11647,-130,803,...,NaN,NaN,49150,924819,338891.0,5.0,NaN,A seguito della periodica verifica dei casi se...,1211357,180
6,2020-12-30T17:00:00,7,Liguria,712,63,775,4853,5628,-131,415,...,NaN,NaN,59997,706883,322079.0,2.0,I 155891 test antigenici rapidi sono riferiti ...,NaN,1543127,209
7,2020-12-30T17:00:00,8,Emilia-Romagna,2641,230,2871,53186,56057,-755,1427,...,NaN,NaN,169396,2556405,1221526.0,19.0,NaN,NaN,4467118,634
8,2020-12-30T17:00:00,9,Toscana,869,160,1029,8708,9737,-278,460,...,NaN,NaN,119696,1872240,1057968.0,10.0,Positivi diagnosticati solo con test antigenic...,NaN,3722729,523
9,2020-12-30T17:00:00,10,Umbria,256,44,300,3406,3706,138,347,...,NaN,NaN,28660,500003,251329.0,8.0,NaN,NaN,880285,117


In [12]:
# Copy last merged df into df_r
df_r = df_m_r_2.copy()

In [13]:
# Add columns
df_r['variazione_ricoverati_con_sintomi'] = df_r_1['ricoverati_con_sintomi'] - df_r_0['ricoverati_con_sintomi']
df_r['variazione_terapia_intensiva'] = df_r_1['terapia_intensiva'] - df_r_0['terapia_intensiva']
df_r['variazione_totale_ospedalizzati'] = df_r_1['totale_ospedalizzati'] - df_r_0['totale_ospedalizzati']
df_r['posti_disponibili_terapia_intensiva'] = df_r['totale_posti_terapia_intensiva'] - df_r['terapia_intensiva']
df_r['saturazione_terapia_intensiva'] = ((df_r_1['terapia_intensiva'] / df_r['totale_posti_terapia_intensiva']) * 100).round(2)
df_r['variazione_isolamento_domiciliare'] = df_r_1['isolamento_domiciliare'] - df_r_0['isolamento_domiciliare']
df_r['variazione_nuovi_positivi'] = df_r_1['nuovi_positivi'] - df_r_0['nuovi_positivi']
df_r['nuovi_tamponi'] = df_r_1['tamponi'] - df_r_0['tamponi']
df_r['nuovi_casi_testati'] = df_r_1['casi_testati'] - df_r_0['casi_testati']
df_r['nuovi_deceduti'] = df_r_1['deceduti'] - df_r_0['deceduti']
df_r['nuovi_dimessi_guariti'] = df_r_1['dimessi_guariti'] - df_r_0['dimessi_guariti']
df_r['np_su_nt'] = ((df_r['nuovi_positivi'] / df_r['nuovi_tamponi']) * 100).round(2)
df_r['np_su_p'] = ((df_r['nuovi_positivi'] / df_r['popolazione']) * 100).round(5)
df_r['tc_su_p'] = ((df_r['totale_casi'] / df_r['popolazione']) * 100).round(2)

df_r = df_r.sort_values(by='nuovi_positivi', ascending=False).reset_index(drop=True)
df_r

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,saturazione_terapia_intensiva,variazione_isolamento_domiciliare,variazione_nuovi_positivi,nuovi_tamponi,nuovi_casi_testati,nuovi_deceduti,nuovi_dimessi_guariti,np_su_nt,np_su_p,tc_su_p
0,2020-12-30T17:00:00,5,Veneto,2711,358,3069,86954,90023,-342,2986,...,35.24,-367,331,17849,4129.0,112,3216,16.73,0.06084,5.08
1,2020-12-30T17:00:00,3,Lombardia,3617,481,4098,48898,52996,-1422,1673,...,46.43,-1388,830,23878,6791.0,80,3015,7.01,0.01656,4.70
2,2020-12-30T17:00:00,16,Puglia,1497,128,1625,51115,52740,-93,1470,...,34.69,-116,721,11121,4812.0,23,1540,13.22,0.03667,2.23
3,2020-12-30T17:00:00,8,Emilia-Romagna,2641,230,2871,53186,56057,-755,1427,...,36.28,-807,533,18360,4928.0,69,2113,7.77,0.03194,3.79
4,2020-12-30T17:00:00,12,Lazio,2808,303,3111,71352,74463,-222,1333,...,35.77,-174,115,13678,5128.0,66,1489,9.75,0.02273,2.75
5,2020-12-30T17:00:00,19,Sicilia,1085,166,1251,32136,33387,-22,1084,...,28.23,-11,89,8497,5298.0,29,1077,12.76,0.02182,1.86
6,2020-12-30T17:00:00,1,Piemonte,2957,192,3149,25810,28959,-424,1046,...,33.39,-348,206,8967,3170.0,33,1437,11.66,0.02409,4.53
7,2020-12-30T17:00:00,15,Campania,1357,102,1459,76152,77611,-880,930,...,20.20,-840,305,11985,7594.0,47,1763,7.76,0.01607,3.25
8,2020-12-30T17:00:00,6,Friuli Venezia Giulia,641,59,700,10947,11647,-130,803,...,32.78,-115,310,7930,1540.0,15,918,10.13,0.06629,4.06
9,2020-12-30T17:00:00,11,Marche,468,62,530,10042,10572,443,585,...,43.36,439,77,7315,3346.0,11,131,8.00,0.03853,2.70


In [14]:
# Write pre-processed data
df_r.to_csv(csv_out_name, index=False)

In [15]:
# ICU Stats
df_r[['denominazione_regione', 'terapia_intensiva', 'posti_disponibili_terapia_intensiva', 'totale_posti_terapia_intensiva', 'saturazione_terapia_intensiva']] \
    .sort_values(by='saturazione_terapia_intensiva', ascending=False).reset_index(drop=True)

,denominazione_regione,terapia_intensiva,posti_disponibili_terapia_intensiva,totale_posti_terapia_intensiva,saturazione_terapia_intensiva
0,Trentino-Alto Adige,67,39,106,63.21
1,Lombardia,481,555,1036,46.43
2,Marche,62,81,143,43.36
3,Umbria,44,73,117,37.61
4,Emilia-Romagna,230,404,634,36.28
5,Lazio,303,544,847,35.77
6,Veneto,358,658,1016,35.24
7,Puglia,128,241,369,34.69
8,Piemonte,192,383,575,33.39
9,Friuli Venezia Giulia,59,121,180,32.78


In [16]:
def add_cols(df):
    df['variazione_terapia_intensiva'] = df['terapia_intensiva'].diff()
    df['variazione_ricoverati_con_sintomi'] = df['ricoverati_con_sintomi'].diff()
    df['variazione_totale_ospedalizzati'] = df['totale_ospedalizzati'].diff()
    df['nuovi_dimessi_guariti'] = df['dimessi_guariti'].diff()
    df['nuovi_deceduti'] = df['deceduti'].diff()
    df['nuovi_tamponi'] = df['tamponi'].diff()
    df['nuovi_casi_testati'] = df['casi_testati'].diff()
    df['variazione_isolamento_domiciliare'] = df['isolamento_domiciliare'].diff()
    df = df.iloc[1:]
    df['ti_su_to'] = ((df['terapia_intensiva'] / df['totale_ospedalizzati']).replace(np.inf, 0) * 100).round(2)
    df['to_su_tp'] = ((df['totale_ospedalizzati'] / df['totale_positivi']).replace(np.inf, 0) * 100).round(2)
    df['np_su_nt'] = ((df['nuovi_positivi'] / df['nuovi_tamponi']).replace(np.inf, 0) * 100).round(2)
    df['np_su_nct'] = ((df['nuovi_positivi'] / df['nuovi_casi_testati']).replace(np.inf, 0) * 100).round(2)
    df['var_ti_perc'] = ((df['variazione_terapia_intensiva'] / (df['terapia_intensiva'] - df['variazione_terapia_intensiva'])).replace(np.inf, 0) * 100).round(2)
    df['var_pos_perc'] = ((df['variazione_totale_positivi'] / (df['totale_positivi'] - df['variazione_totale_positivi'])).replace(np.inf, 0) * 100).round(2)
    return df.fillna(0)

In [17]:
# df_rd.sort_values(by='codice_regione').reset_index(drop=True)
df_rn = pd.read_csv(regions_name)
for region_code in df_rn.codice_regione:
    if region_code != 4: # TAA
        df_cur_region = df_rd[df_rd.codice_regione == region_code]
        region_name = df_cur_region.denominazione_regione.iloc[0].lower()
        csv_out_region_name = '{path_csv}/regioni/covid19-{region_name}.csv'.format(path_csv=path_csv, region_name=region_name)
        df_cur_region = add_cols(df_cur_region)
        df_cur_region.to_csv(csv_out_region_name, index=False)
        print('Aggiornato {file}'.format(file=csv_out_region_name))

Aggiornato ../data/csv/regioni/covid19-piemonte.csv
Aggiornato ../data/csv/regioni/covid19-valle d'aosta.csv
Aggiornato ../data/csv/regioni/covid19-lombardia.csv
Aggiornato ../data/csv/regioni/covid19-veneto.csv
Aggiornato ../data/csv/regioni/covid19-friuli venezia giulia.csv
Aggiornato ../data/csv/regioni/covid19-liguria.csv
Aggiornato ../data/csv/regioni/covid19-emilia-romagna.csv
Aggiornato ../data/csv/regioni/covid19-toscana.csv
Aggiornato ../data/csv/regioni/covid19-umbria.csv
Aggiornato ../data/csv/regioni/covid19-marche.csv
Aggiornato ../data/csv/regioni/covid19-lazio.csv
Aggiornato ../data/csv/regioni/covid19-abruzzo.csv
Aggiornato ../data/csv/regioni/covid19-molise.csv
Aggiornato ../data/csv/regioni/covid19-campania.csv
Aggiornato ../data/csv/regioni/covid19-puglia.csv
Aggiornato ../data/csv/regioni/covid19-basilicata.csv
Aggiornato ../data/csv/regioni/covid19-calabria.csv
Aggiornato ../data/csv/regioni/covid19-sicilia.csv
Aggiornato ../data/csv/regioni/covid19-sardegna.csv


In [18]:
df_tb = df_rd[df_rd.codice_regione > 20].reset_index(drop=True)
df_tb

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,note_test,note_casi
0,2020-02-24T18:00:00,21,P.A. Bolzano,0,0,0,0,0,0,0,0,0,NaN,NaN,0,1,NaN,NaN,NaN,NaN
1,2020-02-24T18:00:00,22,P.A. Trento,0,0,0,0,0,0,0,0,0,NaN,NaN,0,3,NaN,NaN,NaN,NaN
2,2020-02-25T18:00:00,21,P.A. Bolzano,1,0,1,0,1,1,1,0,0,NaN,NaN,1,1,NaN,NaN,NaN,NaN
3,2020-02-25T18:00:00,22,P.A. Trento,0,0,0,0,0,0,0,0,0,NaN,NaN,0,3,NaN,NaN,NaN,NaN
4,2020-02-26T18:00:00,21,P.A. Bolzano,1,0,1,0,1,0,0,0,0,NaN,NaN,1,1,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
617,2020-12-28T17:00:00,22,P.A. Trento,374,45,419,1402,1821,46,99,18413,914,NaN,NaN,21148,443360,150162.0,2.0,NaN,"Dei 99 casi odierni, 25 sono molecolari con pr..."
618,2020-12-29T17:00:00,21,P.A. Bolzano,136,22,158,10520,10678,71,91,17607,729,NaN,NaN,29014,357337,161485.0,3.0,NaN,NaN
619,2020-12-29T17:00:00,22,P.A. Trento,380,46,426,1334,1760,-61,175,18637,926,NaN,NaN,21323,445114,150638.0,4.0,NaN,"Dei 175 casi odierni, 83 sono molecolari con p..."
620,2020-12-30T17:00:00,21,P.A. Bolzano,168,22,190,10412,10602,-76,220,17901,731,NaN,NaN,29234,359695,162050.0,1.0,NaN,NaN


In [19]:
# Initialize df_taa
df_taa = Functions.replace_bt_with_taa(df_tb[:2])
for i in range(2, df_tb.shape[0], 2):
    df_taa = df_taa.append(Functions.replace_bt_with_taa(df_tb[i:i+2]), ignore_index=True)
df_taa = add_cols(df_taa)
df_taa

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,nuovi_deceduti,nuovi_tamponi,nuovi_casi_testati,variazione_isolamento_domiciliare,ti_su_to,to_su_tp,np_su_nt,np_su_nct,var_ti_perc,var_pos_perc
1,2020-02-25T18:00:00,4,Trentino-Alto Adige,1,0,1,0,1,1,1,...,0.0,0.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.00
2,2020-02-26T18:00:00,4,Trentino-Alto Adige,1,0,1,0,1,0,0,...,0.0,0.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.00
3,2020-02-27T18:00:00,4,Trentino-Alto Adige,1,0,1,0,1,0,0,...,0.0,30.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.00
4,2020-02-28T18:00:00,4,Trentino-Alto Adige,1,0,1,0,1,0,0,...,0.0,0.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.00
5,2020-02-29T17:00:00,4,Trentino-Alto Adige,1,0,1,0,1,0,0,...,0.0,25.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
306,2020-12-26T17:00:00,4,Trentino-Alto Adige,486,65,551,11741,12292,-49,148,...,12.0,1723.0,-500.0,-79.0,11.80,4.48,8.59,-29.60,-4.41,-0.40
307,2020-12-27T17:00:00,4,Trentino-Alto Adige,510,65,575,11780,12355,63,143,...,7.0,1376.0,337.0,39.0,11.30,4.65,10.39,42.43,0.00,0.51
308,2020-12-28T17:00:00,4,Trentino-Alto Adige,527,65,592,11836,12428,73,219,...,14.0,1850.0,552.0,56.0,10.98,4.76,11.84,39.67,0.00,0.59
309,2020-12-29T17:00:00,4,Trentino-Alto Adige,516,68,584,11854,12438,10,266,...,19.0,2852.0,654.0,18.0,11.64,4.70,9.33,40.67,4.62,0.08


In [20]:
region_name = df_taa.denominazione_regione.iloc[0].lower()
csv_out_region_name = '{path_csv}/regioni/covid19-{region_name}.csv'.format(path_csv=path_csv, region_name=region_name)
df_taa.to_csv(csv_out_region_name, index=False)
print('Aggiornato {file}'.format(file=csv_out_region_name))

Aggiornato ../data/csv/regioni/covid19-trentino-alto adige.csv
